In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import ast


# 1. 加载主数据集
data = pd.read_csv("CarGroup/0.csv",header=None)
data

In [ ]:



# 定义列名
column_names = [
    'vin','数据采集时间', '车辆状态', '充电状态', '运行模式', '车速', '累积里程', '总电压', '总电流',
    'SOC', 'DC-DC状态', '档位', '绝缘电阻', '经度', '纬度', '电池单体电压最高值',
    '电池单体电压最低值', '最高温度值', '最低温度值', '驱动电机序号', '驱动电机转速', '驱动电机转矩'
]

# 插入列名
data.columns = column_names

# 2. 主数据预处理
# 处理数值数据的有效范围和异常值，删除包含不符合要求的值的行
# data['车速'] = data['车速'].apply(lambda x: x * 0.1 if 0 <= x <= 2200 else None)  # 有效范围0~2200
# data['累积里程'] = data['累积里程'].apply(lambda x: x * 0.1 if 0 <= x <= 9999999 else None)  # 有效范围0~9999999
# data['总电压'] = data['总电压'].apply(lambda x: x * 0.1 if 0 <= x <= 10000 else None)  # 有效范围0~10000
# data['总电流'] = data['总电流'].apply(lambda x: (x - 1000) * 0.1 if 0 <= x <= 20000 else None)  # 有效范围-1000A~1000A
# data['SOC'] = data['SOC'].apply(lambda x: x if 0 <= x <= 100 else None)  # 有效范围0~100
# data['绝缘电阻'] = data['绝缘电阻'].apply(lambda x: x * 1000 if 0 <= x <= 60000 else None)  # 有效范围0~60000
# data['电池单体电压最高值'] = data['电池单体电压最高值'].apply(lambda x: x * 0.001 if 0 <= x <= 15000 else None)  # 有效范围0~15000
# data['电池单体电压最低值'] = data['电池单体电压最低值'].apply(lambda x: x * 0.001 if 0 <= x <= 15000 else None)  # 有效范围0~15000
# data['最高温度值'] = data['最高温度值'].apply(lambda x: x - 40 if 0 <= x <= 250 else None)  # 有效范围-40°C~+210°C
# data['最低温度值'] = data['最低温度值'].apply(lambda x: x - 40 if 0 <= x <= 250 else None)  # 有效范围-40°C~+210°C
# data['驱动电机转速'] = data['驱动电机转速'].apply(lambda x: x - 20000 if 0 <= x <= 65531 else None)  # 有效范围-20000r/min~45531r/min
# data['驱动电机转矩'] = data['驱动电机转矩'].apply(lambda x: (x - 2000) * 0.1 if 0 <= x <= 65531 else None)  # 有效范围-2000N·m~4553.1N·m


# # 将主数据中包含方括号的字符串转换为数值列表
# data['驱动电机序号'] = data['驱动电机序号'].apply(lambda x: ast.literal_eval(x))
# data['驱动电机转速'] = data['驱动电机转速'].apply(lambda x: ast.literal_eval(x))
# data['驱动电机转矩'] = data['驱动电机转矩'].apply(lambda x: ast.literal_eval(x))

# # 遍历来获得所有列的唯一值
# for column in data.columns:
#     unique_values = data[column].unique()
#     print(f"Column {column} unique values: {unique_values}")

data

In [ ]:
data2 = pd.read_csv("trip_data.csv")
data2 = data2[:25]

# 筛选出第二个数据集中 trip_kind 列值为 'D' 的数据
filtered_data2 = data2[data2['trip_kind'] == 'D']

# 初始化一个空的DataFrame，用于存储筛选后的结果
final_data = pd.DataFrame(columns=data.columns)

# 遍历第一个数据集
for index, row in data.iterrows():
    vin = row['vin']
    collection_time = pd.to_datetime(row['数据采集时间'])

    # 查找是否有相同 vin 值的数据在第二个数据集中
    matching_data2 = filtered_data2[filtered_data2['vin'] == vin]

    # 遍历匹配的数据
    for _, matching_row in matching_data2.iterrows():
        start_time = pd.to_datetime(matching_row['start_collectiontime'])
        end_time = pd.to_datetime(matching_row['end_collectiontime'])

        # 如果数据采集时间在 start_time 和 end_time 之间，将该行添加到结果DataFrame
        if start_time <= collection_time <= end_time:
            matching_row['index'] = matching_row['unnamed']  # 添加'index'列
            final_data = final_data.append(matching_row, ignore_index=True)

# 打印筛选后的结果
print(final_data)

In [ ]:
pd.to_csv(final_data,"final_data.csv")